# **Convolutional Neural Network Architectures for Image Analysis**

**Objectives:**
- Understand how to setup a convolutional neural network (CNN) through 5 steps: architecture setup, optimisation setup, training, validation and testing
- Understand the residual neural network (ResNet) architecture
- Understand the U-net architecture

**Why can't we just use a dense NN to processing images?**

Dense neural net's are not ideal for processing images. Firstly, image data has to be flattened to a very long vector of pixel values. This means we need many weights and biases. A 100x100 RBG image would have an input dimension of 30,000! Addionally, the forward pass would take a long time and effectively training such a large model (with lots of learnable parameters) requires a large amount of data. Also neural net's cannot  capture the spatial relationships within an image. 

**What is a CNN?**

A CNN is a deep learning algorithm designed to process and analyse visual data, such as images. It automatically learns patterns and features from an image by applying filters with learnable weights and biases. These filters detect important features like edges, textures, shapes and eventaully entire objects. One of the key strengths of CNNs is their ability to recognise spatial hierarchies of features. This means the network learns to detect simple patterns (e.g edges) in the early layers, then combines these patterns to identify more complex structures (e.g. corners and textures) in deeper layers, and finally recognises entire objects (e.g. faces, cars). This step-by-step feature extraction allows CNN's to understand images in a way that mimics how humans percieve what they see. CNNs can automatically learn which features are important, making them highly effective for tasks such as **image classification**, **facial recognition** and **object detection**.

*Here is a visualisation of a convolutional neural network:* 

<div style="text-align: center;">
    <img src="images/cnn_visualisation.png" width="750" height="250">
</div>

### Input Images

Images are represented as **3D tensors** (multi-dimensional arrays), structured as a stack of 2D colour channels. The most commonly used colour model is the RGB colour model, where there are 3 channels - red, green and blue. Each pixel is defined by 3 values, representing the intensity, of each colour, where 0 is the lowest intensity and 255 is the highest intensity. The picture on the left shows a 4x4x3 RGB image, where the height and width are 4 pixels each, and the depth represents the 3 colour channels. Note, there can be more than 3 colour channels (C > 3). The picture on the right represents a general image tensor.

<div style="display: flex; justify-content: center; gap: 10px;">
    <img src="images/rgb_grid.png" width="400" height="300">
    <img src="images/image_tensor.png" width="400" height="300">
</div>

### Convolution Layer

The purpose of the convolution layer is to **extract the features of an image** such as edges, from the input image. CNNs may have multple convolutional layers. Conventionally, the first convolutional layer extracts low level features such as edges, color, gradient orientation etc. With added layers, the architecture adapts to the high-level features as well, such as faces or objects.

During training, a CNN tries to learn what features in needs to solve the classification (or other) problem. The kernels in a convolutional layer determine what kinds of features it creates. Therefore, **during training, we learn the kernels of the convolutional layer**.

**Kernel**

A kernel in a convolution layer is a small matrix of weights that act like a filter, sliding across an input image to extract specific local features by performing element-wise multiplication with the corresponding pixels in the image (convolution). It essentially identifies patterns within the data.

For example, say the kernel/filter is,

$$
K =
\begin{bmatrix}
1 & 0 & 1 \\
0 & 1 & 0 \\
1 & 0 & 1
\end{bmatrix}
$$


Consider a 5x5x1 image, representing a single-channel image with 5x5 pixels. For simplicity, let's assume pizel values are either 0 or 1, though in reality, they can range from 0 to 255. We apply the 3x3x1 kernel defined above. This kernel will slide across the image, performing convolution at each step. The images below illustrate this process, with the left side showing the kernel's movement and the right showing the resulting convolved feature.

Note, the **depth of a kernel in a CNN corresponds to the depth of the input tensor**. If the input data is an RBG image (C = 3), then the kernel will also have a depth of 3.

**Stride Length**

The stride length refers to the number of pixels the kernel moves by when sliding over the image. In the example below, the stride length is set to 1.

**Convolution Operation**

<div style="display: grid; grid-template-columns: repeat(3, 1fr); gap: 10px; max-width: 600px;">
    <img src="images/conv_layer_1.png" alt="Image 1" style="width: 100%; height: auto;">
    <img src="images/conv_layer_2.png" alt="Image 2" style="width: 100%; height: auto;">
    <img src="images/conv_layer_3.png" alt="Image 3" style="width: 100%; height: auto;">
    <img src="images/conv_layer_4.png" alt="Image 4" style="width: 100%; height: auto;">
    <img src="images/conv_layer_5.png" alt="Image 5" style="width: 100%; height: auto;">
    <img src="images/conv_layer_6.png" alt="Image 6" style="width: 100%; height: auto;">
    <img src="images/conv_layer_7.png" alt="Image 7" style="width: 100%; height: auto;">
    <img src="images/conv_layer_8.png" alt="Image 8" style="width: 100%; height: auto;">
    <img src="images/conv_layer_9.png"alt="Image 9" style="width: 100%; height: auto;">
</div>

- 1 kernel/filter produces 1 feature map
- Each kernel detects a specific feature
- 1 convolutional layer typically has multiple kernels

**Movement of Kernel**

<div style="text-align: center;">
    <img src="images/kernel_movement.png" width="280" height="270">
</div>

**How does this process extract important features?**

Consider the following kernel.
$$
K = 
\begin{bmatrix}
-1 & -1 & -1 \\
0 & 0 & 0 \\
1 & 1 & 1
\end{bmatrix}
$$


- -1 means that pixel intensity is subtracted from the top part.
- +1 means that pixel intensity is added from the top part

This results in a high convolution value at places where there's a strong intensity difference. So, if an image has a sharp transition (like from dark to light), the kernel outputs a high value, highlighting an edge!

**Padding**

This convolution operation can have one of 2 results:
1) Convolved feature has a decreased dimensionality compared to the input image -> done using **valid padding**
2) Convolved feature has the same or increased dimensionality compared to the input image -> done using **same padding**.

*Valid Padding* -> If we apply a 3x3x1 kernel over a 5x5x1 image, the convolved matrix will have size 3x3x1 (like the example above). We can see that the dimensionality has decreased. 

*Same Padding* -> If we augement the 5x5x1 image into a 6x6x1 image, and then apply the 3x3x1 kernel over it, the convolved matrix will have size 5x5x1. We can see that the dimensionality has stayed the same. 

<div style="text-align: center;">
    <img src="images/zero_padding.png" width="280" height="270">
</div>



### Activations - ReLU (Rectified Linear Unit)

The convolution layer produces a set of feature maps that capture various patterns in the input image (e.g edges, textures). After the convolutional layer, the next step involves aplying an activation function, like a ReLU, to introduce non-linearity into the network. This allows the netwrok to learn more complex non-linear patterns and features in the data. It also highlights the features learned by the convolution and filters out less useful information (like the negative activations in the ReLu).

ReLU works by replacing all negative values in the feature map with 0 and keeping all positive values.

$$
\text{ReLU}(x) = \max{(0, x)}
$$

### Pooling Layer
The function of the pooling layer is to **reduce the spatial dimensions** of the feature maps (after the activation function is applied). This is to **decrease the required computational power** to process the data. It is also useful for **extracting dominant features** which are rotational and positional invariant. 

This process works by using a filter (also known as a pooling window) to process small regions og the feature map, typically size 2x2 or 3x3. The filter slides over the feature map with a defined strid (typically 2). For each region, it preforms pooling.

There are 2 types of pooling:
1) Max pooling -> returns the maximum value 
2) Average pooling -> returns the average of all values 

Typically, max pooling performs a lot better than average pooling.

*Note, a pooling filter is typically not the same size and stride as a convolution filter.*

<div style="display: flex; justify-content: center; gap: 10px;">
    <img src="images/conv_and_pooling.png" width="600" height="300">
    <img src="images/pooling.png" width="400" height="300">
</div>

### Fully Connected Layer

A dense neural network is used after the convolutional layers in a CNN to learn non-linear combinations of the high-level features extracted by the convolutional operations. In this stage, the feature maps generated by the convolutional layers are flattened into a one-dimensional vector. This vector is then passed through the FCNN. The network can learn complex patterns and help me the model make more accurate predictions. By using a softmax activation function in the output layer, the network can classify the image based on the learned features, distinguishing between dominant and low-level characteristics to detemine the most likely class.